In [1]:
import warnings
warnings.filterwarnings("ignore")
import wrangle as w

import pandas as pd
import numpy as np

#splits
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

#scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, classification_report

#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
import kc_wrangle as w
import wrangle as ww


# Acquire

In [2]:
df = pd.read_csv("merged_data - merged_data.csv")

In [3]:
df = ww.get_prep_aa(df)

* lowercase variables
* intake age/ outcome age leave just days
* name column made need to be dropped from too many nulls 
* animal id column change to id 


# Prepare

In [4]:
#find numeric columns
num_col = df.select_dtypes(include='number').columns
num_col

Index(['intake_age', 'outcome_age'], dtype='object')

In [5]:
#get string columns
str_col = ['outcome', 'intake_type', 'intake_condition',
       'species', 'found_location', 'intake_sex', 'breed', 'color']

In [6]:
df.intake_sex.value_counts()

intact male      57597
intact female    53284
neutered male    39091
spayed female    30913
unknown           4723
nan                  2
Name: intake_sex, dtype: int64

In [7]:
df.intake_condition.value_counts()

normal        165607
injured         8562
sick            5116
nursing         3849
neonatal         863
aged             538
other            365
medical          297
feral            142
pregnant         135
behavior          96
med attn          16
neurologic         7
med urgent         5
space              4
unknown            4
parvo              3
panleuk            1
Name: intake_condition, dtype: int64

rename intake_conditions: Normal, Injured, Immediate attn needed, mental, delayed/terminal

In [8]:
df.head()

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,found_location,intake_sex,breed,color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,austin (tx),neutered male,domestic shorthair mix,brown tabby/white
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,7201 levander loop in austin (tx),intact male,chihuahua shorthair mix,white/brown
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,dove dr and e stassney in austin (tx),intact male,anatol shepherd/labrador retriever,buff
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,austin (tx),neutered male,anatol shepherd/labrador retriever,buff
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,nursing,0,0,cat,12034 research in austin (tx),intact male,domestic shorthair mix,orange tabby


In [9]:
# lowercase everything
df = df.apply(lambda x: x.astype(str).str.lower())

In [10]:
def transform_intake_condition(df):
    df = df.apply(lambda x: x.astype(str).str.lower())
    # Change 'Feral', 'Neurologic', 'Behavior', 'Space' to 'mental' category
    df['intake_condition'] = df['intake_condition'].replace(['feral', 'neurologic', 'behavior', 'space'], 'mental')
    
    # Set values indicating medical attention
    df['intake_condition'] = df['intake_condition'].replace(['nursing', 'neonatal', 'medical', 'pregnant', 'med attn', 'med urgent', 'parvo', 'agonal', 'panleuk'], 'medical attention')
    
    # Drop rows with 'other', 'unknown', and 'nan' values
    df = df[df['intake_condition'].isin(['other', 'unknown', 'nan']) == False]
    
    return df

In [11]:
df = w.transform_intake_condition(df)

# top ten colors and other for erybody else, or for simplify all colors

In [12]:
def transform_color(df):
    # lowercase everything
    df = df.apply(lambda x: x.astype(str).str.lower())

    
    # Add spaces between color names separated by slashes
    df['color'] = df['color'].str.replace('/', ' / ')
    
    # Replace color names with their corresponding standard names
    replacements = {
        'chocolate': 'brown',
        'liver': 'brown',
        'ruddy': 'brown',
        'apricot': 'orange',
        'pink': 'red',
        'cream': 'white',
        'flame point': 'white',
        'blue': 'gray',
        'silver': 'gray',
        'yellow': 'gold',
        'torbie': 'tricolor',
        'tortie': 'tricolor',
        'calico': 'tricolor'
    }
    df['color'] = df['color'].replace(replacements, regex=True)
    
    # Create new column 'primary_color' with the first color
    colors = ['black', 'brown', 'white', 'tan', 'brindle', 'gray', 'fawn', 'red', 'sable', 'buff', 'orange', 'blue', 'tricolor', 'gold', 'cream', 'lynx point', 'seal point', 'agouti', 'lilac point']
    for color in colors:
        df.loc[df['color'].str.startswith(color), 'primary_color'] = color
    
    # Drop rows with 'unknown' color
    df = df[df['color'] != 'unknown']
    
    # Create column indicating if the animal has a tabby pattern
    df['is_tabby'] = df['color'].str.contains('tabby').astype(int)
    
    # Create column indicating if the animal has mixed colors
    df["mix_color"] = np.where(df['color'].str.contains(r'\/|tricolor|torbie|tortie'), 1, 0)
    
    return df




In [13]:
df = w.transform_color(df)

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion